In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import datetime
import math
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
Customer_df=pd.read_csv('/kaggle/input/onlineretail/OnlineRetail.csv',dtype={'CustomerID': str,'InvoiceID': str})
Customer_df.head()

In [ ]:
Customer_df.describe()

In [ ]:
#removing negative values and replacing them with zero
Customer_df.loc[Customer_df['Quantity']<0,'Quantity']=0
Customer_df.loc[Customer_df['UnitPrice']<0,'UnitPrice']=0

In [ ]:
Customer_df[Customer_df.isnull().any(axis=1)]

In [ ]:
# drop rows with null values 
Customer_df=Customer_df.dropna(how='any',axis=0)
#Customer_df[Customer_df.isna().any(axis=1)]

In [ ]:
Customer_df.info()


In [ ]:

Customer_df.InvoiceDate = pd.to_datetime(Customer_df.InvoiceDate)
#calculate the amount spent in each transaction
Customer_df['amount'] = Customer_df.Quantity*Customer_df.UnitPrice
# reference date is one day after transaction day so that the minimum recency is 1
refrence_date = Customer_df.InvoiceDate.max() + datetime.timedelta(days = 1)


In [ ]:
# recency is the number of days since the reference date
Customer_df['recency'] = (refrence_date - Customer_df.InvoiceDate).astype('timedelta64[D]')
# recency per customer, finding the most recent purchase by the customer
Customer_Recency_df = Customer_df[['CustomerID', 'recency']].groupby("CustomerID").min().reset_index()
Customer_Recency_df.describe().transpose()

In [ ]:
#number of purchases per customer. group by invoice number because there might be multiple rows per customer per invoice.
Customer_freq = (Customer_df[['CustomerID', 'InvoiceNo']].groupby(["CustomerID", 'InvoiceNo']).count().reset_index()).\
                groupby(["CustomerID"]).count().reset_index()
Customer_freq.rename(columns={'InvoiceNo':'frequency'},inplace=True)


In [ ]:
# amount: monetary value the amount spend by each customer
Customer_monetary_val = Customer_df[['CustomerID', 'amount']].groupby("CustomerID").sum().reset_index()
Customer_RFM =Customer_Recency_df.merge(Customer_freq).merge(Customer_monetary_val)
Customer_RFM.loc[Customer_RFM['amount']==0,'amount']=0.0001
Customer_RFM.describe()

In [ ]:
# data is skewed so log transform and standard scaler helps to normalize it. 
Customer_RFM['recency_log'] = Customer_RFM['recency'].apply(math.log)
Customer_RFM['frequency_log'] =Customer_RFM['frequency'].apply(math.log)
Customer_RFM['amount_log'] =Customer_RFM['amount'].apply(math.log)
scaler = StandardScaler()
scaled_features = scaler.fit_transform(Customer_RFM[['amount_log', 'recency_log','frequency_log']])


In [ ]:

 
# used 4 clusters here but the number of clusters should be determined based on the Silhouette analysis
kmeans = KMeans(n_clusters=4, init="random", n_init=10, max_iter=300,random_state=42)
kmeans.fit(scaled_features)

# join the clusters to the main data
Customer_RFM['clusters_4'] = kmeans.labels_
cluster_centers=kmeans.cluster_centers_

In [ ]:
display(Customer_RFM.head())
features = ['amount',  'recency',  'frequency']
scaler.inverse_transform(kmeans.cluster_centers_)
data_inversed = scaler.inverse_transform(cluster_centers)
data_transformed=pd.DataFrame(np.exp(data_inversed),columns=features)
data_transformed.reset_index(inplace=True)
data_transformed.rename(columns={"index": "Cluster"},inplace=True)

In [ ]:
sns.set_style("darkgrid")
fig = plt.figure(figsize=(20,7))
f1 = fig.add_subplot(131)
sns.set(font_scale =2)
ax1=sns.boxplot(x=Customer_RFM['clusters_4'],y=Customer_RFM['amount'])
ax1.set_title('Monetary value', fontsize = 25)
ax1.set_ylabel('Amount ($)', fontsize =25)
ax1.set_xlabel ('Cluster', fontsize = 25)
ax1.set_ylim(top=20000,bottom=0)

f1 = fig.add_subplot(132)
sns.set(font_scale = 2)
ax2=sns.boxplot(x=Customer_RFM['clusters_4'],y=Customer_RFM[ 'recency'])
ax2.set_title('Recency', fontsize =25)
ax2.set_ylabel('Recency  (days)', fontsize =25)
ax2.set_xlabel ('Cluster', fontsize = 25)
ax2.set_ylim(top=500,bottom=0)

f1 = fig.add_subplot(133)
sns.set(font_scale = 2)
ax3=sns.boxplot(x=Customer_RFM['clusters_4'],y=Customer_RFM[ 'frequency'])
ax3.set_title('Frequency', fontsize = 25)
ax3.set_ylabel('Frequency', fontsize =25)
ax3.set_xlabel ('Cluster', fontsize = 25)
ax3.set_ylim(top=75,bottom=0)


Cluster 0 is hibernating. Last purchase was long back, low spenders and low number of orders.  Action: Offer other relevant products and special discounts. Recreate brand value.

Cluster 1 are promising customers. Bought recently but didn’t spend much. Action: Create brand awareness, offer deals.

Cluster 2 are the champions. This group buy often, buy recently and spend the most. Action: Reward them. They can be early adopters of new products, and they can promote your brand. This group would be a good target for cross-sell/upsell.

Cluster 3 made big purchases, and often, but not recently! Win them back via newer products, don’t lose them to competition. 
